# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pandas.io.json import json_normalize
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
CityData=pd.read_csv("../output_data/CityDataStructured.csv")
CityData.head()

,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,San Patricio,40,MX,1580090341,74,19.22,-104.70,80.60,9.17
1,1,Weligama,96,LK,1580090341,74,5.97,80.42,77.61,4.61
2,2,Tasiilaq,100,GL,1580090341,50,65.61,-37.64,28.40,2.24
3,3,Bredasdorp,67,ZA,1580089914,76,-34.53,20.04,55.40,1.12
4,4,Chokurdakh,98,RU,1580090006,96,70.63,147.92,-26.28,4.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = CityData[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#CityData= CityData.dropna()
HumidityValue = CityData["Humidity"].astype(float)
maxHumidity=CityData["Humidity"].max()

In [5]:
# Create a Humidiy Heatmap layer
myfig = gmaps.figure()

HeatLayer = gmaps.heatmap_layer(locations, weights=HumidityValue, 
                                 dissipating=False, max_intensity=maxHumidity,
                                 point_radius = 1)
myfig.add_layer(HeatLayer)
myfig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
CityDataFresh=CityData.dropna(how="all")

In [58]:
###### METHOD 1 ###### My Direct ###########()
MyFavoritePlaces=CityDataFresh.loc[(CityDataFresh["Cloudiness"] == 0) & (CityDataFresh["Wind Speed"] < 10) & ((CityDataFresh["Max Temperature"] >70 ) & (CityDataFresh["Max Temperature"] <= 80))]
MyFavoritePlaces

,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Cloud Feel,Temp Feel,Wind Feel
177,177,Jacmel,0,HT,1580090362,78,18.23,-72.53,77.00,4.70,Favorable,Favorable,Favorable
204,204,Narasannapeta,0,IN,1580090365,56,18.42,84.05,74.07,4.81,Favorable,Favorable,Favorable
224,224,Warri,0,NG,1580090368,86,5.52,5.75,71.11,2.01,Favorable,Favorable,Favorable
239,239,Barra de Tecoanapa,0,MX,1580090370,85,16.52,-98.75,74.30,1.61,Favorable,Favorable,Favorable
466,466,Salaga,0,GH,1580090397,11,8.55,-0.52,72.86,2.95,Favorable,Favorable,Favorable
500,500,Pozo Colorado,0,PY,1580090401,36,-23.49,-58.80,79.25,2.64,Favorable,Favorable,Favorable
528,528,Srikakulam,0,IN,1580090406,56,18.30,83.90,74.07,4.81,Favorable,Favorable,Favorable
540,540,Diu,0,IN,1580090408,52,20.71,70.98,72.54,8.93,Favorable,Favorable,Favorable
548,548,Maxixe,0,MZ,1580090410,81,-23.86,35.35,77.40,9.98,Favorable,Favorable,Favorable


In [8]:
###### METHOD 2 ###### My Binning ###########
WeatherCriteria_Bin_Build=[CityDataFresh["Cloudiness"].describe()["min"],0.1,CityDataFresh["Cloudiness"].describe()["25%"],CityDataFresh["Cloudiness"].describe()["50%"],CityDataFresh["Cloudiness"].describe()["75%"],CityDataFresh["Cloudiness"].describe()["max"]]
WeatherCriteria_Bin =["Favorable","Clear Sky","Mild Cloud","Mostly Cloudy","Totally Cloudy"]
Temp_Bin_Build=[CityDataFresh["Max Temperature"].describe()["min"],70,80,CityDataFresh["Max Temperature"].describe()["max"]]
Temp_Bin=["Cold","Favorable","Hot"]
Wind_Bin_Build=[CityDataFresh["Wind Speed"].describe()["min"],10,CityDataFresh["Wind Speed"].describe()["max"]]
Wind_Bin=["Favorable","Windy"]
CityData_Favor = CityDataFresh
CityData_Favor["Cloud Feel"] = pd.cut(CityData_Favor["Cloudiness"], WeatherCriteria_Bin_Build ,right=True,labels=WeatherCriteria_Bin,include_lowest=True)
CityData_Favor["Temp Feel"]= pd.cut(CityData_Favor["Max Temperature"], Temp_Bin_Build ,right=True,labels=Temp_Bin,include_lowest=True)
CityData_Favor["Wind Feel"]= pd.cut(CityData_Favor["Wind Speed"], Wind_Bin_Build ,right=True,labels=Wind_Bin,include_lowest=True)
CityData_Favor.sort_values(['Wind Speed'], ascending=[1])
BestWeatherCities=CityData_Favor.loc[(CityData_Favor["Wind Feel"] == "Favorable") & (CityData_Favor["Temp Feel"] == "Favorable") & (CityData_Favor["Cloud Feel"] == "Favorable")]
len(BestWeatherCities)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
hotel_df=MyFavoritePlaces

In [67]:
HotelName = []
HotelAddress=[]
# function to extract hotel name
def GetHotelName(Latitude,Longitude):
    TargetRadius = 5000
    TargetType = "Hotel"
    TargetCoordinates =  Latitude + " , " + Longitude 
    params = {
    "location": TargetCoordinates,
    "radius": TargetRadius,
    "type": TargetType,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    resp = requests.get(base_url, params=params)
    rtext = json.loads(resp.text)
    response = json_normalize(rtext)
    ResultedHotels=json_normalize(response["results"][0])
    HotelName.append(ResultedHotels["name"][1])
    HotelAddress.append(ResultedHotels["vicinity"][1])

In [68]:
for index, row in BestWeatherCities.iterrows():
    GetHotelName(str(row['Latitude']),str(row['Longitude']))

In [70]:
HotelName,HotelAddress

(['Hotel Florita',
  'Mahesh Traders',
  'Warri Township Stadium',
  'UTMarGRO',
  'Presbyterian Church',
  'Planta de tratamiento de agua potable',
  'Jeans Park',
  'Hotel Relax Inn',
  'Oceano'],
 ['Rue Du Commerce, Jacmel',
  'Narasannapeta Road, Maruthi Nagar, Narasannapeta',
  'Warri',
  'Marquelia',
  'Salaga',
  'Pozo Colorado',
  'GT Road, Chinna Bharatham Colony, Srikakulam',
  'Bunder Rd, Diu',
  'Maxixe'])

In [72]:
hotel_df.insert(2, "Hotel Name", HotelName, True) 
hotel_df.insert(3, "Hotel Address", HotelAddress, True)
hotel_df

,City ID,City Name,Hotel Name,Hotel Address,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Cloud Feel,Temp Feel,Wind Feel
177,177,Jacmel,Hotel Florita,"Rue Du Commerce, Jacmel",0,HT,1580090362,78,18.23,-72.53,77.00,4.70,Favorable,Favorable,Favorable
204,204,Narasannapeta,Mahesh Traders,"Narasannapeta Road, Maruthi Nagar, Narasannapeta",0,IN,1580090365,56,18.42,84.05,74.07,4.81,Favorable,Favorable,Favorable
224,224,Warri,Warri Township Stadium,Warri,0,NG,1580090368,86,5.52,5.75,71.11,2.01,Favorable,Favorable,Favorable
239,239,Barra de Tecoanapa,UTMarGRO,Marquelia,0,MX,1580090370,85,16.52,-98.75,74.30,1.61,Favorable,Favorable,Favorable
466,466,Salaga,Presbyterian Church,Salaga,0,GH,1580090397,11,8.55,-0.52,72.86,2.95,Favorable,Favorable,Favorable
500,500,Pozo Colorado,Planta de tratamiento de agua potable,Pozo Colorado,0,PY,1580090401,36,-23.49,-58.80,79.25,2.64,Favorable,Favorable,Favorable
528,528,Srikakulam,Jeans Park,"GT Road, Chinna Bharatham Colony, Srikakulam",0,IN,1580090406,56,18.30,83.90,74.07,4.81,Favorable,Favorable,Favorable
540,540,Diu,Hotel Relax Inn,"Bunder Rd, Diu",0,IN,1580090408,52,20.71,70.98,72.54,8.93,Favorable,Favorable,Favorable
548,548,Maxixe,Oceano,Maxixe,0,MZ,1580090410,81,-23.86,35.35,77.40,9.98,Favorable,Favorable,Favorable


In [73]:
# Create a Hotels
# Store 'Lat' and 'Lng' into  locations 
hlocations = hotel_df[["Latitude", "Longitude"]].astype(float)
HumidityValue = CityData["Humidity"].astype(float)
maxHumidity=CityData["Humidity"].max()

myhotellocator = gmaps.figure()

HotelHeatLayer = gmaps.heatmap_layer(locations, weights=HumidityValue, 
                                 dissipating=False, max_intensity=maxHumidity,
                                 point_radius = 1)

myhotellocator.add_layer(HotelHeatLayer)
myhotellocator

Figure(layout=FigureLayout(height='420px'))

In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [79]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
MyInfoTemplate = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [80]:
# Add marker layer ontop of heat map
Hover=np.array(hotel_df["City Name"])
HotelMarker_Layer=gmaps.marker_layer(hlocations,info_box_content=MyInfoTemplate,display_info_box=True)
myhotellocator.add_layer(HotelMarker_Layer)


# Display Map
myhotellocator

Figure(layout=FigureLayout(height='420px'))